In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud

pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
from fractions import Fraction
import ast

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'width'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=2426

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

# from natsort import natsorted
# .apply(lambda x: natsorted(x))

# def atof(text):
#     try:
#         retval = float(text)
#     except ValueError:
#         retval = text
#     return retval

# def natural_keys(text):
#     '''
#     alist.sort(key=natural_keys) sorts in human order
#     http://nedbatchelder.com/blog/200712/human_sorting.html
#     (See Toothy's implementation in the comments)
#     float regex comes from https://stackoverflow.com/a/12643073/190597
#     '''
#     return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

# #remove duplicates
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']]

['This is a fraction: 100.02']

# Keyword Processor for Text

In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

7067

#### Regex for fraction, decimal, and integer

In [7]:
regex_pattern=r'(?i)(\d+\.\d+.{0,3}?[LWDH]\b)|(\d+.?\d+\/\d+.{0,3}?[LWDH]\b)|((?<!\.)(?<!\/)\d+(?!\.\d).{0,3}?[LWDH]\b)|((?<!\.)(?<!\/)\d+(?!\.).{0,3}?[LWDH]\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|()' #fraction, decimal, int
def regex_patterns(unit):
    return fr'(?i)((?<!\.)\d+.?\d+\/\d+.?.?.?{unit}\b)|((?<!\\.)\d+\.\d+.?.?.?{unit}\b)|((?<![\.\/])\d+.?.?.?{unit}\b)|()'
print(regex_patterns('Le?n?g?t?h?'))

(?i)((?<!\.)\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\\.)\d+\.\d+.?.?.?Le?n?g?t?h?\b)|((?<![\.\/])\d+.?.?.?Le?n?g?t?h?\b)|()


In [8]:
def regex_patterns(unit):
    return fr'(?i)((?<!\\.)\\d+.?\\d+\\/\\d+.?.?.?{unit}\\b)|((?<!\\.)\\d+\\.\\d+.?.?.?{unit}\\b)|((?<![\.\/])\d+.?.?.?{unit}\b)|()'
regex_patterns('W(?!eigh)')

'(?i)((?<!\\\\.)\\\\d+.?\\\\d+\\\\/\\\\d+.?.?.?W(?!eigh)\\\\b)|((?<!\\\\.)\\\\d+\\\\.\\\\d+.?.?.?W(?!eigh)\\\\b)|((?<![\\.\\/])\\d+.?.?.?W(?!eigh)\\b)|()'

In [9]:
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
# natural_sort['matches']=natural_sort['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))

In [10]:
x=str(["20 in","1-1/2 in","54 in","60 in","66 in","68 in","72 in","78 in","81 in","90 in","108 in"])
remove_duplicates([n.strip() for n in ast.literal_eval(x)])
# [list(set(sublist)) for sublist in [n.strip() for n in ast.literal_eval(x)]]

['20 in',
 '1-1/2 in',
 '54 in',
 '60 in',
 '66 in',
 '68 in',
 '72 in',
 '78 in',
 '81 in',
 '90 in',
 '108 in']

In [11]:
regex_pattern=r'(?i)(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df['matches']=df['matches'].apply(lambda x: remove_duplicates(x))
df['matches']=df['matches'].apply(lambda x: natsorted(x))

In [12]:
df['matches'].head(2)

0           []
1    [7.5\" W]
Name: matches, dtype: object

In [14]:
natural_sort=df
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub(r'\\" W|"W',' in',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
natural_sort['matches'].head(2)

0          
1    7.5 in
Name: matches, dtype: object

In [15]:
natural_sort['matches']=natural_sort['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
natural_sort['matches'].head(2)

0          []
1    [7.5 in]
Name: matches, dtype: object

In [16]:
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x)))                                                                          
natural_sort['matches'].head(2)

0       [" in"]
1    ["7.5 in"]
Name: matches, dtype: object

In [17]:
natural_sort['matches']=natural_sort['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])) # for duplicate numbers
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']] # for duplicate text
natural_sort['matches'][0:500]

0                                      [in]
1                                  [7.5 in]
2                                   [10 in]
3                                   [10 in]
4                                      [in]
5                                   [10 in]
6                                  [6.4 in]
7                                      [in]
8                                   [16 in]
9                                   [18 in]
10                                  [18 in]
11                       [16.3 in, 16.9 in]
12                       [16.3 in, 16.9 in]
13                                [23.6 in]
14                                [23.6 in]
15                                [20.1 in]
16                                [20.1 in]
17                                [21.7 in]
18                                  [18 in]
19                                  [14 in]
20                                  [12 in]
21                                     [in]
22                             [

In [ ]:
natural_sort[curation_col]=natural_sort['matches']
matches_natural=natural_sort[['external_id',curation_col]]
matches_natural

In [ ]:
# def remove_frac(numbers):
#     if str(numbers).find(',') > -1:  
#         nom, den = numbers.split('/')
#         tmp = float(nom)/float(den)
#         return tmp
#     else:
#         return numbers

In [78]:
# df_a = pd.DataFrame({"A": [[16.0, 24.4175], [14.9687, 16.06], [22.75, 23.00]]})
# # df_a = pd.DataFrame({"A": [[16, 24.42], [14.97, 16.06], [22.75, 23]]})
# df_a

In [76]:
# df_a.applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])

In [77]:
# x=[16.00, 24.4175]
# [round_string_float(num) for num in x]

In [69]:
# df_a = pd.DataFrame({"A": [[16.0, 24.4175], [14.9687, 16.06], [22.75, 23.00]]})
# df_a.applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])

A
0     [16, 24.42]
1  [14.97, 16.06]
2     [22.75, 23]

In [20]:
# natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# # natural_sort['matches']=natural_sort['matches'].apply(lambda x: remove_frac(remove_fraction(x)))
# # natural_sort['matches']=natural_sort['matches'].apply(lambda x: [round_string_float(num) for num in x])
# x=pd.DataFrame(natural_sort['matches']).applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])
# # natural_sort['matches'].apply(lambda x: [round(v, None if float(v).is_integer() else 2) for v in x])
# list = []
# for i in range(len(x)):
#     for j in range(len(x[i])):
#         list .append(round(x[i][j]))
# x

In [10]:
df_new=df_len[df_len['comma_matches'].astype(str)=='[]']
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_new['matches']=df_new['matches'].apply(lambda x: remove_frac(remove_fraction(x)))
df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))

In [11]:
df_new['matches']=df_new['matches'].apply(lambda x:str(x)+' in')

In [12]:
df_new[curation_col]=df_new['matches']
matches=df_new[['external_id',curation_col]]
matches

external_id  Q:width
1        62866903    18 in
2        69763719    18 in
3        69763717    18 in
5        42595717    13 in
6        61535404     7 in
...           ...      ...
50261    69400100     3 in
50266    66882459  27.5 in
50272    64215396   117 in
50273    61608726   117 in
50274    61608733   117 in

[11652 rows x 2 columns]

In [10]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

[]

In [12]:
#--------------------------------------------------------------------------------------------
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df[df['matches'].astype(str)!='[]']

None


Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, matches]
Index: []

In [13]:
metric=" Inches"
range_string =""""
13 - 17 
Greater than 18
3 - 12
"""
# range_string =""""

#  Greater than 108
#  12 - 23.9
#  24 - 35.9
#  36 - 47.9
#  48 - 59.9
#  60 - 71.9
#  72 - 83.9
#  84 - 95.9
#  96 - 107.9
#  0 - 11.9
#  """

In [14]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower} to {upper}{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        no_words=original_val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return f'{no_words}{metric} and Higher'
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >= 13 and num <= 17 :
        return f'13 to 17  Inches'
        
    if num >= 18:
        return f'18 Inches and Higher'
        
    if num >= 3 and num <= 12:
        return f'3 to 12 Inches'
        
    else:
        return r"n/a"



In [15]:
matches[curation_col]=matches['matches'].apply(range_app)
matches.head()

NameError: name 'matches' is not defined

# send to the folder for upload

In [18]:
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-multiples.csv',index=False) 

In [19]:
looks_good('Bed Bath & Beyond', attribute, df, matches_natural)